In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import GradientBoostingRegressor

pd.options.display.max_rows = 20
pd.options.display.max_columns = 100

train_data_file = '../input/train.csv'
type_dictionary = '../input/data_types.csv'
test_data_file = '../input/test.csv'

In [2]:
type_dict = {}
with open(type_dictionary) as f:
    reader = csv.reader(f)
    for row in reader:
        type_dict[row[0]] = row[1]

In [3]:
train = pd.read_csv(train_data_file, dtype=type_dict)
train_raw = train.copy()
train_id = train.Id
train.drop(columns='Id', inplace=True)

In [4]:
test = pd.read_csv(test_data_file, dtype=type_dict)
test_raw = test.copy()
test_id = test.Id
test.drop(columns='Id', inplace=True)

In [5]:
def fill_missing_data(dfin):
    df = dfin.copy()
    categoricals = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu',
                    'GarageCond','GarageType','GarageFinish','GarageQual',
                   'BsmtExposure','BsmtFinType2','BsmtFinType1','BsmtCond','BsmtQual']
    for column in categoricals:
        df.loc[:,column].cat.add_categories(['None'], inplace=True)
        df.loc[:,column].fillna('None', inplace=True)
    df.MasVnrType.fillna('None', inplace=True)
    
    floats = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath','BsmtUnfSF',
              'GarageArea', 'GarageCars', 'TotalBsmtSF','MasVnrArea']
    for column in floats:
        df.loc[:,column].fillna(0, inplace=True)
        
    df.Exterior1st.fillna('VinylSd', inplace=True)
    df.Exterior2nd.fillna('VinylSd', inplace=True)
    df.Functional.fillna('Typ', inplace=True)
    df.KitchenQual.fillna('TA', inplace=True)
    df.MSZoning.fillna('RL', inplace=True)
    df.SaleType.fillna('WD', inplace=True)
    df.Utilities.fillna('AllPub', inplace=True)
    df.Electrical.fillna('SBrkr', inplace=True)
    
    df.LotFrontage.fillna(df.LotFrontage.median(), inplace=True)
    df.GarageYrBlt.fillna(9999, inplace=True)
    
    return df

In [33]:
def clean_categoricals(dfin):
    df = dfin.copy()
    
    #This will set the proper order for all of the ordinal variables
    df.BsmtCond.cat.set_categories(['None','Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.BsmtExposure.cat.set_categories(['None','No','Mn','Av','Gd'], ordered=True, inplace=True)
    df.BsmtFinType1.cat.set_categories(['None','Unf','LwQ','Rec','BLQ','ALQ','GLQ'], ordered=True, inplace=True)
    df.BsmtFinType2.cat.set_categories(['None','Unf','LwQ','Rec','BLQ','ALQ','GLQ'], ordered=True, inplace=True)
    df.BsmtQual.cat.set_categories(['None','Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.Electrical.cat.set_categories(['Mix','FuseP','FuseF','FuseA','SBrkr'], ordered=True, inplace=True)
    df.ExterCond.cat.set_categories(['Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.ExterQual.cat.set_categories(['Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.Fence.cat.set_categories(['None','MnWw','GdWo','MnPrv','GdPrv'], ordered=True, inplace=True)
    df.FireplaceQu.cat.set_categories(['None','Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.Functional.cat.set_categories(['Sal','Sev','Maj2','Maj1','Mod','Min2','Min1','Typ'], ordered=True, inplace=True)
    df.GarageCond.cat.set_categories(['None','Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.GarageFinish.cat.set_categories(['None','Unf','RFn','Fin'], ordered=True, inplace=True)
    df.GarageQual.cat.set_categories(['None','Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.HeatingQC.cat.set_categories(['Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.KitchenQual.cat.set_categories(['Po','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.LandSlope.cat.set_categories(['Sev','Mod','Gtl'], ordered=True, inplace=True)
    df.LotShape.cat.set_categories(['Reg','IR1','IR2','IR3'], ordered=True, inplace=True)
    df.OverallCond.cat.set_categories(['1','2','3','4','5','6','7','8','9','10'], ordered=True, inplace=True)
    df.OverallQual.cat.set_categories(['1','2','3','4','5','6','7','8','9','10'], ordered=True, inplace=True)
    df.PavedDrive.cat.set_categories(['N','P','Y'], ordered=True, inplace=True)
    df.PoolQC.cat.set_categories(['None','Fa','TA','Gd','Ex'], ordered=True, inplace=True)
    df.Utilities.cat.set_categories(['ELO','NoSeWa','NoSewr','AllPub'], ordered=True, inplace=True)
    
    # This will set which categories from the nominal variables will be used (NaNs are created)
    df.MSSubClass.cat.set_categories(['20','50','60','Other'], inplace=True)
    df.MSZoning.cat.set_categories(['RL','RM','Other'], inplace=True)
    df.LandContour.cat.set_categories(['Lvl','Other'], inplace=True)
    df.LotConfig.cat.set_categories(['Inside','Other'], inplace=True)
    df.Condition1.cat.set_categories(['Artery', 'Feedr', 'Norm', 'PosA', 'PosN','Other'], inplace=True)
    df.Condition2.cat.set_categories(['Norm','Other'], inplace=True)
    df.BldgType.cat.set_categories(['1Fam','Other'], inplace=True)
    df.HouseStyle.cat.set_categories(['1.5Fin', '1Story', '2Story', 'SLvl', 'Other'], inplace=True)
    df.RoofStyle.cat.set_categories(['Gable','Flat','Hip','Other'], inplace=True)
    df.RoofMatl.cat.set_categories(['CompShg','Other'], inplace=True)
    df.Exterior1st.cat.set_categories(['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd',
       'HdBoard', 'ImStucc', 'MetalSd', 'Plywood', 'Stone', 'Stucco',
       'VinylSd', 'Wd Sdng', 'WdShing'], inplace=True)
    df.Exterior2nd.cat.set_categories(['AsbShng', 'AsphShn', 'Brk Cmn', 'BrkFace', 'CBlock', 'CmentBd',
       'HdBoard', 'ImStucc', 'MetalSd', 'Other', 'Plywood', 'Stone', 'Stucco',
       'VinylSd', 'Wd Sdng', 'Wd Shng'], inplace=True)
    df.Heating.cat.set_categories(['Floor', 'GasA', 'Other'], inplace=True)
    df.GarageType.cat.set_categories(['Attchd','Detchd','None','Other'], inplace=True)
    df.MiscFeature.cat.set_categories(['Shed','None','Other'], inplace=True)
    df.SaleCondition.cat.set_categories(['Normal','Other'], inplace=True)
    
    #This fills the NaNs into the recently created 'Other' category
    others = ['MSSubClass', 'MSZoning', 'LandContour', 'LotConfig', 'Condition1',
              'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
               'Heating', 'GarageType', 'MiscFeature', 'SaleCondition']
    for col in others:
        df[col].fillna('Other', inplace=True)
    
    return df

In [7]:
def drop_colinear(dfin):
    df = dfin.copy()
    columns_to_drop = ['GarageYrBlt', 'GrLivArea', 'TotalBsmtSF', 'GarageArea', 'YearRemodAdd']
    return df.drop(columns_to_drop, axis=1)

In [8]:
def apply_power_transform(dfin):
    df = dfin.copy()
    pt = PowerTransformer(copy=False)
    floats = df.dtypes=='float64'
    data = df.loc[:,floats].values
    pt.fit_transform(data)
    transformed_data = pd.DataFrame(data, columns=df.loc[:,floats].columns)
    for col in transformed_data.columns:
        df[col] = transformed_data[col]
    return df

In [9]:
def all_dummies(df):
    dummies = df.copy()
    ordinals = ['BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtQual',
               'Electrical','ExterCond','ExterQual','Fence','FireplaceQu',
               'Functional','GarageCond','GarageFinish','GarageQual','HeatingQC',
               'KitchenQual','LandSlope','LotShape','OverallCond','OverallQual',
               'PavedDrive','PoolQC','Utilities']
    for col in ordinals:
        dummies[col] = dummies[col].cat.codes
    return pd.get_dummies(dummies, drop_first=True)

In [10]:
def transform_data(dfin):
    df = dfin.copy()
    df_clean = fill_missing_data(df)
    df_clean = clean_categoricals(df_clean)
    df_clean = drop_colinear(df_clean)
    df_clean = apply_power_transform(df_clean)
    df_clean = all_dummies(df_clean)
    
    return df_clean

In [11]:
def fit_predict(regressor, X_train, Y_train, X_test, rname):
    regressor.fit(X_train, Y_train)
    Y_pred = regressor.predict(X_test)
    submission = pd.DataFrame({'Id':test_id.astype(int), 'SalePrice':Y_pred})
    submission.to_csv("../output/" + rname + ".csv", index=False)

In [34]:
X_train = train.iloc[:,:-1].copy()
Y_train = train.iloc[:,-1].copy()

X_train_clean = transform_data(X_train)
X_test_clean = transform_data(test)

In [18]:
grad_boost = GradientBoostingRegressor()
grad_boost.fit(X_train_clean, Y_train)
scores = cross_val_score(grad_boost, X_train_clean, Y_train, cv=10)
print('CV accuracy (original): %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))
highest_score = np.mean(scores)

for i in range(1, X_train_clean.shape[1]+1, 1):
    # Select i features
    select = SelectKBest(score_func=f_regression, k=i)
    select.fit(X_train_clean, Y_train)
    X_train_clean_selected = select.transform(X_train_clean)

    # Model with i features selected
    grad_boost.fit(X_train_clean_selected, Y_train)
    scores = cross_val_score(grad_boost, X_train_clean_selected, Y_train, cv=10)
    print('CV accuracy (number of features = %i): %.3f +/- %.3f' % (i, 
                                                                     np.mean(scores), 
                                                                     np.std(scores)))
    
    # Save results if best score
    if np.mean(scores) > highest_score:
        highest_score = np.mean(scores)
        std = np.std(scores)
        k_features_highest_score = i
    elif np.mean(scores) == highest_score:
        if np.std(scores) < std:
            highest_score = np.mean(scores)
            std = np.std(scores)
            k_features_highest_score = i
        
# Print the number of features
print('Number of features when highest score: %i' % k_features_highest_score)

CV accuracy (original): 0.876 +/- 0.050
CV accuracy (number of features = 1): 0.675 +/- 0.034
CV accuracy (number of features = 2): 0.685 +/- 0.029
CV accuracy (number of features = 3): 0.686 +/- 0.038
CV accuracy (number of features = 4): 0.735 +/- 0.036
CV accuracy (number of features = 5): 0.766 +/- 0.035
CV accuracy (number of features = 6): 0.769 +/- 0.041
CV accuracy (number of features = 7): 0.788 +/- 0.039
CV accuracy (number of features = 8): 0.807 +/- 0.072
CV accuracy (number of features = 9): 0.804 +/- 0.076
CV accuracy (number of features = 10): 0.827 +/- 0.068
CV accuracy (number of features = 11): 0.826 +/- 0.071
CV accuracy (number of features = 12): 0.826 +/- 0.069
CV accuracy (number of features = 13): 0.833 +/- 0.062
CV accuracy (number of features = 14): 0.827 +/- 0.065
CV accuracy (number of features = 15): 0.823 +/- 0.071
CV accuracy (number of features = 16): 0.825 +/- 0.053
CV accuracy (number of features = 17): 0.826 +/- 0.053
CV accuracy (number of features = 

CV accuracy (number of features = 149): 0.874 +/- 0.046
CV accuracy (number of features = 150): 0.871 +/- 0.053
CV accuracy (number of features = 151): 0.877 +/- 0.047
CV accuracy (number of features = 152): 0.874 +/- 0.053
CV accuracy (number of features = 153): 0.872 +/- 0.052
CV accuracy (number of features = 154): 0.877 +/- 0.048
CV accuracy (number of features = 155): 0.874 +/- 0.051
CV accuracy (number of features = 156): 0.874 +/- 0.049
Number of features when highest score: 89


In [36]:
select = SelectKBest(score_func=f_regression, k=k_features_highest_score)
select.fit(X_train_clean, Y_train)
X_train_clean_selected = select.transform(X_train_clean)
X_test_clean_selected = select.transform(X_test_clean)

In [37]:
grad_boost_ls = GradientBoostingRegressor()
fit_predict(grad_boost_ls, X_train_clean_selected, Y_train, X_test_clean_selected, "grad_boost_feature selected")